In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import datetime

from __future__ import print_function

%matplotlib inline

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/MNIST", one_hot=True)

In [ ]:
image_size = mnist.train.images.shape[1]

In [ ]:
batch_size = 128
noise_size = 100

In [ ]:
def create_weights_and_biases(units_per_layer):
    W = []
    b = []
    for i in range(1, len(units_per_layer)):
        W.append(tf.get_variable("W" + str(i), [units_per_layer[i - 1], units_per_layer[i]], dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer(seed=1)))
        b.append(tf.get_variable("b" + str(i), [units_per_layer[i]], dtype=tf.float32,
            initializer=tf.zeros_initializer()))
        
    theta = []
    theta.extend(W)
    theta.extend(b)
    
    return W, b, theta

In [ ]:
def discriminator(input_image, W, b):
    Z1 = tf.add(tf.matmul(input_image, W[0], name="W1xA0"), b[0], name="Z1")
    A1 = tf.nn.relu(Z1, name="A1")
    Z2 = tf.add(tf.matmul(A1, W[1], name="W2xA1"), b[1], name="Z2")
    A2 = tf.nn.sigmoid(Z2, name="A2")

    return A2

In [ ]:
def generator(input_noise):
    W, b, theta = create_weights_and_biases([noise_size, 128, image_size])
    
    Z1 = tf.add(tf.matmul(input_noise, W[0], name="W1xA0"), b[0], name="Z1")
    A1 = tf.nn.relu(Z1, name="A1")
    Z2 = tf.add(tf.matmul(A1, W[1], name="W2xA1"), b[1], name="Z2")
    A2 = tf.nn.sigmoid(Z2, name="A2")

    return A2, theta

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=[None, image_size], name="X")
Z = tf.placeholder(dtype=tf.float32, shape=[None, noise_size], name="Z")

with tf.variable_scope("G"):
    G, G_theta = generator(Z)
    
with tf.variable_scope("D"):
    D_W, D_b, D_theta = create_weights_and_biases([image_size, 128, 1])
    
    with tf.variable_scope("D_X"):
        D_X = discriminator(X, D_W, D_b)
        
    with tf.variable_scope("D_G"):
        D_G = discriminator(G, D_W, D_b)

with tf.variable_scope("D_optimizer"):
    D_loss = -tf.reduce_mean(tf.log(D_X) + tf.log(1.0 - D_G))
    D_optimizer = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_theta)

with tf.variable_scope("G_optimizer"):
    G_loss = -tf.reduce_mean(tf.log(D_G))
    G_optimizer = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_theta)

In [ ]:
def sample_noise(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [ ]:
sample_size = 16
sample_grid = int(np.ceil(np.sqrt(sample_size)))

In [ ]:
def plot_samples(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(sample_grid, sample_grid)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis("off")
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect("equal")
        plt.imshow(sample.reshape(28, 28), cmap="Greys_r")
        
    plt.show()
    plt.pause(0.001)

    return fig

In [ ]:
iterations = 100000
log_every = 10000

plt.ion()

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    for iteration in range(iterations):
        batch_X, _ = mnist.train.next_batch(batch_size)
        batch_noise_D = sample_noise(batch_size, noise_size)
        batch_noise_G = sample_noise(batch_size, noise_size)

        _, _ = session.run([D_loss, D_optimizer], feed_dict={X: batch_X, Z: batch_noise_D})

        batch_noise_G = sample_noise(batch_size, noise_size)

        _, _ = session.run([G_loss, G_optimizer], feed_dict={Z: batch_noise_G})

        if iteration % log_every == log_every - 1:
            print(iteration + 1)
            samples, = session.run([G], feed_dict={Z: sample_noise(sample_size, noise_size)})
            plot_samples(samples)